# Import and Preprocess Initial Data

#### Team members: Shengzhao LEI - Tao SUN -  Xiangzhe MENG

With this notebook, we import all the initial datasets, drop all the duplicates rows and save the new dataframes as pickle files in order to reduce the data loading time.

For this project, we choose 4 sub-datasets from the [amazon review dataset](http://jmcauley.ucsd.edu/data/amazon/links.html):

1. Kindle store metadata
2. Kindle store 5-core reviews
3. Books metadata
4. Books 5-core reviews

In [3]:
%matplotlib inline
import pandas as pd
import gzip
import json
import numpy as np
import matplotlib.pyplot as plt

### 1- Functions for loading data

We use the two functions below to import data from **gz** extention file. These two functions are provided in the website of our Amazon dataset.

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

In [3]:
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

### 2 - Functions for checking Nan value

We define the two functions below to verify if there exists nan value in the chosen column.

In [4]:
def checkNanValue(dataframe, column):
    print("Check if column {} exists Nan value: {}".format(column,dataframe[column].isnull().values.any()))
    
def checkDataframeNanValue(dataframe):
    list_columns = list(dataframe.columns)
    for column in list_columns:
        checkNanValue(dataframe, column)

### 3 - Import Kindle store metadata

In [69]:
df_kindle_meta = getDF('data/meta_Kindle_Store.json.gz')

In [70]:
df_kindle_meta

,asin,description,price,imUrl,related,categories,title,salesRank,brand
0,1603420304,In less time and for less money than it takes ...,7.69,http://ecx.images-amazon.com/images/I/51IEqPrF...,"{'also_viewed': ['B001OLRKLQ', 'B004J35JIC', '...","[[Books, Cookbooks, Food & Wine, Quick & Easy]...",NaN,NaN,NaN
1,B0002IQ15S,This universal DC adapter powers/charges porta...,19.99,http://ecx.images-amazon.com/images/I/21QFJM28...,"{'also_viewed': ['B00511PS3C', 'B000PI17MM', '...","[[Kindle Store, Kindle Accessories, Power Adap...",Mobility IGO AUTOPOWER 3000 SERIES ( PS0221-10 ),{},NaN
2,B000F83SZQ,NaN,0.00,http://ecx.images-amazon.com/images/I/51yLqHe%...,"{'also_bought': ['B0080H1C0W', 'B00LK4ZKOG', '...","[[Books, Literature & Fiction], [Books, Myster...",NaN,NaN,NaN
3,B000F83TEQ,NaN,NaN,http://ecx.images-amazon.com/images/I/2136NBNV...,"{'also_bought': ['B00IS81LFO', 'B000FA5T6A', '...","[[Books, Literature & Fiction], [Books, Myster...",NaN,NaN,NaN
4,B000F83STC,NaN,NaN,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN,"[[Books, Literature & Fiction, Erotica], [Kind...",NaN,NaN,NaN
5,B000FA5RE4,Updated 2003 version with corrections,1.99,http://ecx.images-amazon.com/images/I/21XTM6V0...,"{'also_viewed': ['B001GX3ZWA', 'B009GI3S1A', '...","[[Books, Reference, Dictionaries & Thesauruses...",NaN,NaN,NaN
6,B000FA5NSO,NaN,NaN,http://ecx.images-amazon.com/images/I/51N45B6j...,"{'also_viewed': ['B000FBF81K', 'B000FA5KKA']}","[[Books, Literature & Fiction], [Books, Scienc...",NaN,NaN,NaN
7,B000FA5UXC,Madoc Tamlin is a man with an unusual problem....,NaN,http://ecx.images-amazon.com/images/I/51q4iur5...,"{'also_bought': ['B004SPL1I8', 'B00942QL3K', '...","[[Books, Literature & Fiction], [Books, Scienc...",NaN,NaN,NaN
8,B000FA5T4W,NaN,12.46,http://ecx.images-amazon.com/images/I/51eWyBr8...,"{'also_bought': ['B004RKXHGS', 'B004ULMJH2', '...","[[Books, Business & Money, Accounting], [Books...",NaN,NaN,NaN
9,B000FA5SHK,NaN,NaN,http://ecx.images-amazon.com/images/I/51c7mqOR...,NaN,"[[Books, Science & Math, Behavioral Sciences],...",NaN,NaN,NaN


From the metadata, we can know that there are a lot of null values and the dataset contains not only information about kindle books but also about kindle accessories. We have to do some data cleaning work on this dataset.

#### 3.1 - Discard several useless columns

We discard **salesrank & brand** because all the values in these columns are related to the kindle accessories instead of kindle books and we cannot get any information about salesrank & brand about kindle books. As for **description, imUrl, related, title**, we drop them as well because they are not quite useful for our project and also there is little useful information related to kindle books in these columns. For example, you cannot find the titles of any kindle books.

In [7]:
df_kindle_meta = df_kindle_meta.drop(['description','imUrl','related','title','salesRank','brand'],axis=1)

#### 3.2 - Check if there exits NaN Value in the DataFrame

In [8]:
checkDataframeNanValue(df_kindle_meta)

Check if column asin exists Nan value: False
Check if column price exists Nan value: True
Check if column categories exists Nan value: False


#### 3.3 - Replace Nan value by 0

According to the result above, we know that only column **price** exists Nan value. Thus, we decide to replace **Nan** values as **0**, which also makes sense because there are tons of free kindle books in Amazon.

In [9]:
df_kindle_meta['price'] = df_kindle_meta['price'].fillna(0)

#### 3.4 - Only keep ebook products and discard all the other relative kindle products like accessories

In this part, we **drop all the kindle accessories products** from the dataset with the help of the attribute **category**.

In [10]:
rows_to_delete = []
for idx, categories in enumerate(df_kindle_meta['categories']):
    isBook = False
    for category in categories[0]:
        if 'Books' in category:
            isBook = True
            break
    if isBook == False:
        rows_to_delete.append(idx)

In [11]:
df_kindle_meta = df_kindle_meta.drop(df_kindle_meta.index[rows_to_delete])

#### 3.5 - Set asin as index

In [12]:
df_kindle_meta = df_kindle_meta.set_index(['asin'])

#### 3.6 - Generate pickle file

In [13]:
df_kindle_meta.to_pickle('kindle_metadata.pkl')

### 4 - Import book metadata
The structure of book metadata is similar to that of kindle metadata. But since there are only books in the dataset, so, we don't need to delete unrelated products. In this part, we import book metadata and we just apply the same procedure as above before to generate the pickle file.

In [79]:
df_book_meta = getDF('data/meta_Books.json.gz')

In [85]:
df_book_meta = df_book_meta.set_index('asin')

#### 4.1 - Discard several useless columns from the DateFrame

In [41]:
df_book_meta = df_book_meta.drop(['description','imUrl','related','title','brand'],axis=1)

#### 4.2 - Check if there exists NaN Value in the DataFrame

In [42]:
checkDataframeNanValue(df_book_meta)

Check if column asin exists Nan value: False
Check if column salesRank exists Nan value: True
Check if column categories exists Nan value: False
Check if column price exists Nan value: True


#### 4.3 - Replace NaN value by 'unknown' or 0

In [43]:
df_book_meta['salesRank'] = df_book_meta['salesRank'].fillna('unknown')
df_book_meta['price'] = df_book_meta['price'].fillna(0)

#### 4.4 - Set asin as index

In [44]:
df_book_meta = df_book_meta.set_index('asin')

#### 4.5 - Generate pickle file

In [45]:
df_book_meta.to_pickle('book_metadata.pkl')

### 5 - Import Kindle store reviews

In this part, we begin to turn our attention to the review dataset.

In [71]:
df_kindle_review = getDF('data/reviews_Kindle_Store_5.json.gz')

In [72]:
df_kindle_review.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F6404F1VG29J,B000F83SZQ,Avidreader,"[0, 0]",I enjoy vintage books and movies so I enjoyed ...,5.0,Nice vintage story,1399248000,"05 5, 2014"
1,AN0N05A9LIJEQ,B000F83SZQ,critters,"[2, 2]",This book is a reissue of an old one; the auth...,4.0,Different...,1388966400,"01 6, 2014"
2,A795DMNCJILA6,B000F83SZQ,dot,"[2, 2]",This was a fairly interesting read. It had ol...,4.0,Oldie,1396569600,"04 4, 2014"
3,A1FV0SX13TWVXQ,B000F83SZQ,"Elaine H. Turley ""Montana Songbird""","[1, 1]",I'd never read any of the Amy Brewster mysteri...,5.0,I really liked it.,1392768000,"02 19, 2014"
4,A3SPTOKDG7WBLN,B000F83SZQ,Father Dowling Fan,"[0, 1]","If you like period pieces - clothing, lingo, y...",4.0,Period Mystery,1395187200,"03 19, 2014"


In the review dataset, there are nine columns. 

**reviewerID** and **reviewName** are unrelated to our projects, we choose to delete them. 

Also, we notice there are two attributes about time information, **unixReviewTime** which is only numbers and **reviewTime** which is kind of dirty. Wikipedia tells us that [unix time](https://en.wikipedia.org/wiki/Unix_time) is a system for describing a point in time. Luckily, pandas provides us with function to translate unix format of time to standard format. So, since the format of **reviewTime** is dirty, we choose to keep **unixReviewTime** and delete **reviewTime**. 

#### 5.1 - Discard the column reviewTime which represents the same data as column unixReviewTime

In [75]:
df_kindle_review = df_kindle_review.drop(['reviewerID','reviewerName','reviewTime'],axis=1)

#### 5.2 - Change date format to standard datetime

We convert the unix time format into date time format.

In [76]:
df_kindle_review['unixReviewTime'] = pd.to_datetime(df_kindle_review['unixReviewTime'],unit='s')

#### 5.3 - Check if there exists Nan Value in the DataFrame

There isn't any Nan value in the dataframe. Good!

In [37]:
checkDataframeNanValue(df_kindle_review)

Check if column asin exists Nan value: False
Check if column helpful exists Nan value: False
Check if column reviewText exists Nan value: False
Check if column overall exists Nan value: False
Check if column summary exists Nan value: False
Check if column unixReviewTime exists Nan value: False


#### 5.4 - Generate pickle file

In [38]:
df_kindle_review.to_pickle('kindle_review.pkl')

### 6 - Import book reviews

In [5]:
df_book_review = getDF('data/reviews_Books_5.json.gz')

In [32]:
df_book_review.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5.0,Wonderful!,1355616000,"12 16, 2012"
1,A2S166WSCFIFP5,000100039X,"adead_poet@hotmail.com ""adead_poet@hotmail.com""","[0, 2]",This is one my must have books. It is a master...,5.0,close to god,1071100800,"12 11, 2003"
2,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""","[0, 0]",This book provides a reflection that you can a...,5.0,Must Read for Life Afficianados,1390003200,"01 18, 2014"
3,A1MOSTXNIO5MPJ,000100039X,Alan Krug,"[0, 0]",I first read THE PROPHET in college back in th...,5.0,Timeless for every good and bad time in your l...,1317081600,"09 27, 2011"
4,A2XQ5LZHTD4AFT,000100039X,Alaturka,"[7, 9]",A timeless classic. It is a very demanding an...,5.0,A Modern Rumi,1033948800,"10 7, 2002"


As this dataframe is very large, we can not put all the data in a same pickle file. Thus, we decide to **divide this dataframe into 10 sub-dataframes of same size** and genereate 10 different pickle files.

`review_data_preprocess` is a function for preprocessing review data with the method we discuss above for kinlde review dataset.

In [20]:
def review_data_preprocess(dataFilePath):
    temp = pd.read_pickle(dataFilePath)
    temp = temp.drop(['reviewerID','reviewerName','reviewTime'],axis=1)
    temp['unixReviewTime'] = pd.to_datetime(temp['unixReviewTime'],unit='s')
    temp.to_pickle(dataFilePath)

In [21]:
df_book_review_1000000 = df_book_review.iloc[:1000000]
df_book_review_1000000.to_pickle('book_review_1000000.pkl')
review_data_preprocess('book_review_1000000.pkl')

In [23]:
df_book_review_1000000_2000000 = df_book_review.iloc[1000000:2000000]
df_book_review_1000000_2000000.to_pickle('book_review_1000000_2000000.pkl')
review_data_preprocess('book_review_1000000_2000000.pkl')

In [24]:
df_book_review_2000000_3000000 = df_book_review.iloc[2000000:3000000]
df_book_review_2000000_3000000.to_pickle('book_review_2000000_3000000.pkl')
review_data_preprocess('book_review_2000000_3000000.pkl')

In [25]:
df_book_review_3000000_4000000 = df_book_review.iloc[3000000:4000000]
df_book_review_3000000_4000000.to_pickle('book_review_3000000_4000000.pkl')
review_data_preprocess('book_review_3000000_4000000.pkl')

In [26]:
df_book_review_4000000_5000000 = df_book_review.iloc[4000000:5000000]
df_book_review_4000000_5000000.to_pickle('book_review_4000000_5000000.pkl')
review_data_preprocess('book_review_4000000_5000000.pkl')

In [27]:
df_book_review_5000000_6000000 = df_book_review.iloc[5000000:6000000]
df_book_review_5000000_6000000.to_pickle('book_review_5000000_6000000.pkl')
review_data_preprocess('book_review_5000000_6000000.pkl')

In [28]:
df_book_review_6000000_7000000 = df_book_review.iloc[6000000:7000000]
df_book_review_6000000_7000000.to_pickle('book_review_6000000_7000000.pkl')
review_data_preprocess('book_review_6000000_7000000.pkl')

In [14]:
df_book_review_7000000_8000000 = df_book_review.iloc[7000000:8000000]
df_book_review_7000000_8000000.to_pickle('book_review_7000000_8000000.pkl')
review_data_preprocess('book_review_7000000_8000000.pkl')

In [15]:
df_book_review_8000000 = df_book_review.iloc[8000000:]
df_book_review_8000000.to_pickle('book_review_8000000.pkl')
review_data_preprocess('book_review_8000000.pkl')

### 7 - Drop duplicate rows and update pickle files

#### 7.1 - Read products dataframe and drop duplicate rows

We find out that the information of many kindle books appear not only in the metadata of kindle but also the metadata of books, which makes sense beacause we many books have both two versions. In order to make our pickles cleaner and smaller. We decide to drop all the kindle store products from the book products dataframe on the basis of category and update pickle files. So, in the metadata of kindle, we would have information of books with kindle version, and in the metadata of books, we would have only books.

In [33]:
df_kindle_meta = pd.read_pickle('pickleData/kindle_metadata.pkl')

In [34]:
df_kindle_meta.head()

,price,categories
asin,,
1603420304,7.69,"[[Books, Cookbooks, Food & Wine, Quick & Easy]..."
B000F83SZQ,0.00,"[[Books, Literature & Fiction], [Books, Myster..."
B000F83TEQ,0.00,"[[Books, Literature & Fiction], [Books, Myster..."
B000F83STC,0.00,"[[Books, Literature & Fiction, Erotica], [Kind..."
B000FA5RE4,1.99,"[[Books, Reference, Dictionaries & Thesauruses..."


In [35]:
df_book_meta = pd.read_pickle('pickleData/book_metadata.pkl')

In [36]:
df_book_meta.head()

,salesRank,categories,price
asin,,,
0001048791,{'Books': 6334800},[[Books]],0.00
0001048775,{'Books': 13243226},[[Books]],0.00
0001048236,{'Books': 8973864},[[Books]],9.26
0000401048,{'Books': 6448843},[[Books]],0.00
0001019880,{'Books': 9589258},[[Books]],5.39


In [101]:
for idx in df_kindle_meta.index:
    if idx not in df_book_meta.index:
        print("There exists kindle metadata which is not contained in the book metadata dataset.")
        break

There exists kindle metadata which is not contained in the book metadata dataset.


In [37]:
to_drop = []
for idx in df_kindle_meta.index:
    if idx in df_book_meta.index:
        to_drop.append(idx)

In [38]:
df_book_meta = df_book_meta.drop(to_drop)

#### 7.2 - Read review dataframes and drop duplicate rows

In this part, we clean the review dataframes. There are some books which don't have either metadata or reviews and we need to get those books out of our datasets. We only keep reviews whose related product is in one of the product metadata dataframes and also we only keep products which have reviews in the review dataframes.

In [4]:
df_kindle_review = pd.read_pickle('pickleData/kindle_review.pkl')

In [5]:
df_kindle_review.head()

,asin,helpful,reviewText,overall,summary,unixReviewTime
0,B000F83SZQ,"[0, 0]",I enjoy vintage books and movies so I enjoyed ...,5.0,Nice vintage story,2014-05-05
1,B000F83SZQ,"[2, 2]",This book is a reissue of an old one; the auth...,4.0,Different...,2014-01-06
2,B000F83SZQ,"[2, 2]",This was a fairly interesting read. It had ol...,4.0,Oldie,2014-04-04
3,B000F83SZQ,"[1, 1]",I'd never read any of the Amy Brewster mysteri...,5.0,I really liked it.,2014-02-19
4,B000F83SZQ,"[0, 1]","If you like period pieces - clothing, lingo, y...",4.0,Period Mystery,2014-03-19


In [6]:
def readPickleDataframe(path):
    return pd.read_pickle(path)

In [7]:
FOLDER = 'pickleData/'
df_book_review_1000000 = readPickleDataframe(FOLDER + 'book_review_1000000.pkl')
df_book_review_1000000_2000000 = readPickleDataframe(FOLDER + 'book_review_1000000_2000000.pkl')
df_book_review_2000000_3000000 = readPickleDataframe(FOLDER + 'book_review_2000000_3000000.pkl')
df_book_review_3000000_4000000 = readPickleDataframe(FOLDER + 'book_review_3000000_4000000.pkl')
df_book_review_4000000_5000000 = readPickleDataframe(FOLDER + 'book_review_4000000_5000000.pkl')
df_book_review_5000000_6000000 = readPickleDataframe(FOLDER + 'book_review_5000000_6000000.pkl')
df_book_review_6000000_7000000 = readPickleDataframe(FOLDER + 'book_review_6000000_7000000.pkl')
df_book_review_7000000_8000000 = readPickleDataframe(FOLDER + 'book_review_7000000_8000000.pkl')
df_book_review_8000000 = readPickleDataframe(FOLDER + 'book_review_8000000.pkl')

##### Concatenate 9 sub-dataframes

In [8]:
listToConcatenate = [df_book_review_1000000,df_book_review_1000000_2000000,df_book_review_2000000_3000000,
                     df_book_review_3000000_4000000,df_book_review_4000000_5000000,df_book_review_5000000_6000000,
                     df_book_review_6000000_7000000,df_book_review_7000000_8000000,df_book_review_8000000]
df_book_review = pd.concat(listToConcatenate)

In [9]:
df_book_review.head()

,asin,helpful,reviewText,overall,summary,unixReviewTime
0,000100039X,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5.0,Wonderful!,2012-12-16
1,000100039X,"[0, 2]",This is one my must have books. It is a master...,5.0,close to god,2003-12-11
2,000100039X,"[0, 0]",This book provides a reflection that you can a...,5.0,Must Read for Life Afficianados,2014-01-18
3,000100039X,"[0, 0]",I first read THE PROPHET in college back in th...,5.0,Timeless for every good and bad time in your l...,2011-09-27
4,000100039X,"[7, 9]",A timeless classic. It is a very demanding an...,5.0,A Modern Rumi,2002-10-07


##### Update kindle review dataframe 

In [45]:
df_kindle_review = df_kindle_review.set_index('asin')
# Find mutual asin --> all kindle products which have reviews
kindle_asin_set = list(set(df_kindle_review.index) & set(df_kindle_meta.index))

In [47]:
# Collect all the kindle products to drop
to_drop_meta = list(set(df_kindle_meta.index) - set(kindle_asin_set))
df_kindle_meta = df_kindle_meta.drop(to_drop_meta)

# Collect all the kindle reviews to drop
to_drop_review = list(set(df_kindle_review.index) - set(kindle_asin_set))
df_kindle_review = df_kindle_review.drop(to_drop_review)

In [49]:
# Update kindle products dataframe
df_kindle_meta.to_pickle('pickleDataNew/kindle_metadata.pkl')

# Update kindle reviews dataframe
df_kindle_review.to_pickle('pickleDataNew/kindle_review.pkl')

##### Update book review dataframe

In [53]:
df_book_review = df_book_review.set_index('asin')
# Find mutual asin --> all book products which have reviews
book_asin_set = list(set(df_book_review.index) & set(df_book_meta.index))

In [55]:
# Collect all the book products to drop
to_drop_meta = list(set(df_book_meta.index) - set(book_asin_set))
df_book_meta = df_book_meta.drop(to_drop_meta)

# Collect all the book reviews to drop
to_drop_review = list(set(df_book_review.index) - set(book_asin_set))
df_book_review = df_book_review.drop(to_drop_review)

In [57]:
# Update book products dataframe
df_book_meta.to_pickle('pickleDataNew/book_metadata.pkl')

# Update book reviews dataframe
df_book_review_1000000 = df_book_review.iloc[:1000000]
df_book_review_1000000.to_pickle('pickleDataNew/book_review_1000000.pkl')

df_book_review_1000000_2000000 = df_book_review.iloc[1000000:2000000]
df_book_review_1000000_2000000.to_pickle('pickleDataNew/book_review_1000000_2000000.pkl')

df_book_review_2000000_3000000 = df_book_review.iloc[2000000:3000000]
df_book_review_2000000_3000000.to_pickle('pickleDataNew/book_review_1000000_2000000.pkl')

df_book_review_3000000_4000000 = df_book_review.iloc[3000000:4000000]
df_book_review_3000000_4000000.to_pickle('pickleDataNew/book_review_3000000_4000000.pkl')

df_book_review_4000000_5000000 = df_book_review.iloc[4000000:5000000]
df_book_review_4000000_5000000.to_pickle('pickleDataNew/book_review_4000000_5000000.pkl')

df_book_review_5000000_6000000 = df_book_review.iloc[5000000:6000000]
df_book_review_5000000_6000000.to_pickle('pickleDataNew/book_review_5000000_6000000.pkl')

df_book_review_6000000_7000000 = df_book_review.iloc[6000000:7000000]
df_book_review_6000000_7000000.to_pickle('pickleDataNew/book_review_6000000_7000000.pkl')

df_book_review_7000000 = df_book_review.iloc[7000000:]
df_book_review_7000000.to_pickle('pickleDataNew/book_review_7000000.pkl')

### 8 - What we have done so far

- **First step** -- data cleaning on each dataset:
    - For metadata of kindle store:
        - Delete salesrank, brand, description, imUrl, related, title, 
        - Set all NaN price to 0.
        - Only keep kindle books and drop all others, like accessories.
    - For metadata of book:
        - Delete brand, description, imUrl, related, title
        - Set all NaN price to 0.
        - Set all NaN salesRank to unknown.
    - For review of kindle store:
        - Drop reviewID and reviewName.
        - Drop reviewTime and translate unixReviewTime as new time information
    - For review of kindle store:
        - Drop reviewID and reviewName.
        - Drop reviewTime and translate unixReviewTime as new time information

- **Second Step** -- data cleaning across datasets:
    - Drop duplicated metadata information. From the metadata of books, delete all books whose metadata are already included in the metadata of kindle
    - Drop useless books/kindle books
        - Drop all metadata of books/kindle books which don’t have review in the review dataset.
        - Drop all reviews of books/kindle books which don’t have metadata in the metadata dataset.

